# Simple Mnist Example

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

In [2]:
##### CREATE DATASET #####
def mnist_dataset():
    (x, y), _ = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    
    # take create a dataset with at most arg elements
    ds = ds.take(50000).shuffle(50000).batch(100)
    return ds

def prepare_mnist_features_and_labels(x, y):
    # Casts a tensor to a new type.
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y


In [ ]:
##### CREATE MODEl #####
model = keras.Sequential([
    layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    layers.Dense(100, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(10)])

optimizer = optimizers.Adam()

In [6]:
##### CREATE DATASET #####
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):

    # gradient tape to keep track of the gradients to use with the optimizer
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    # update to weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

def train(epoch, model, optimizer):

    train_ds = mnist_dataset()
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(train_ds):
        loss, accuracy = train_one_step(model, optimizer, x, y)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

    return loss, accuracy


In [7]:
for epoch in range(20):
    loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.278705 ; accuracy 0.13
epoch 1 : loss 0.14803983 ; accuracy 0.96
epoch 2 : loss 0.06116972 ; accuracy 0.99
epoch 3 : loss 0.05639075 ; accuracy 0.99
epoch 4 : loss 0.035583988 ; accuracy 1.0
epoch 5 : loss 0.019163467 ; accuracy 1.0
epoch 6 : loss 0.020616665 ; accuracy 0.99
epoch 7 : loss 0.019613 ; accuracy 1.0
epoch 8 : loss 0.018232979 ; accuracy 1.0
epoch 9 : loss 0.01663007 ; accuracy 1.0
epoch 10 : loss 0.0023701896 ; accuracy 1.0
epoch 11 : loss 0.0177087 ; accuracy 1.0
epoch 12 : loss 0.01740459 ; accuracy 1.0
epoch 13 : loss 0.015479625 ; accuracy 0.99
epoch 14 : loss 0.001899093 ; accuracy 1.0
epoch 15 : loss 0.02245509 ; accuracy 0.99
epoch 16 : loss 0.0013087247 ; accuracy 1.0
epoch 17 : loss 0.0022392091 ; accuracy 1.0
epoch 18 : loss 0.0010316534 ; accuracy 1.0
epoch 19 : loss 0.012830465 ; accuracy 0.99
Final epoch 19 : loss 0.014195996 ; accuracy 1.0
